# Quelques exemples de génération d'article

### Auteur : Boubacar TRAORE, System : Linux

In [14]:
# Changement de la police utilisée et de sa taille
from IPython.core.display import HTML, display
from style import change_font
display(HTML(change_font()))

In [15]:
# Ajout du sommaire
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [5]:
# Importation des libraries nécessaires
import os
import numpy as np

Dans ce notebook, nous nous proposons d'énumérer différentes manières de générer des articles, soit à partir de notre propre corpus (collection de documents d'anciens articles que nous possédons déjà comme base d'entrainement de notre modèle), soit à partir de modèles préentrainés sur un corpus beaucoup plus large.

## 1. Préparation des articles

Les articles que nous utilisons dans cette étude proviennent de l'agence de presse Reuters (https://fr.wikipedia.org/wiki/Reuters). Ces données ont été collectées en 2005. Il s'agit d'un dossier contenant 2500 articles d'actualité (newspaper) venant de 50 auteurs différents. Ces articles sont stockés dans le dossier "**data**".

In [6]:
# Définition des répertoires où sont stockées nos données
data_dir = 'data'   # Spécification du dossier contenant les articles à parser
cwd = os.getcwd() + "/" + data_dir # cwd contient le chemin d'accès vers le dossier 'data'

Nous allons parcourir tous les articles qui se trouvent dans les dossiers nommés d'après leurs auteurs. Pour cela, nous créons deux listes. La première nommée "documents_infos" qui va contenir les informations disponibles de chaque article (titre et nom de l'auteur) sous forme de dictionnaire et la seconde liste nommée "documents" contient le corps de chaque article stockée sous format texte (str). 

In [8]:
documents_infos, documents  = [], []
for folder in os.listdir(cwd):
    for file in os.listdir(cwd + "/" + folder):
        with open(data_dir + '/' + folder + '/' + file, 'r') as f:
            lines = f.readlines()
            infos = {'title': lines[0].strip(), 'author': folder}
            documents_infos.append(infos)
            documents.append(''.join(lines[1:]))

In [9]:
len(documents)

2500

Nous avons identifié 2500 articles. Essayons maintenant d'afficher le contenu (limité aux 500 premiers caractères) et les informations d'un article aléatoire.

In [10]:
np.random.seed(777)
id_doc = np.random.randint(0,len(documents),1)[0]

In [11]:
documents_infos[id_doc]

{'title': 'Fallout from a scandal involving late allocation of trades at Jardine Fleming was still attracting attention in Hong Kong on Wednesday, but the company escaped further sanctions and said business continued as usual.',
 'author': 'SarahDavison'}

In [12]:
print(documents[id_doc][:500] + '....')

"We've had some redemptions. We've also had some subscriptions," a company spokesman told Reuters. "The business pattern is very much as usual."
Jardine Fleming retained its seat on the executive committee of the domestic industry association, the Hong Kong Investment Funds Association, despite media reports that moves were afoot to knock the company off the committee.
In election results released on Wednesday, Jardine Fleming continues as one of 12 member firms with a representative on the boar....


Les articles sont bien stockés. Passons aux différentes méthodes de modélisation.

## 2. Un modèle basique & simple : les chaînes de Markov

Les chaines de Markov est un processus permettant de prédire un évènement suivant en se basant sur l'évènement qui vient tout juste de se réaliser. On dit qu'il s'agit d'un modèle à courte mémoire car toute l'information nécessaire pour prédire le futur est disponible dans le présent et indépendante du futur.

### 2.1. Modèle

Les chaines de Markov peuvent bien être adaptés dans notre cas puisque la génération d'article démarre d'une expression donnée par l'utilisateur, le modèle va donc à chaque fois tenter de deviner le mot suivant à partir du précédent donné. Pour celà, nous pouvons fusionner tous les contenus des articles en un seul grand texte et créer un dictionnaire de mots, qui, à chaque clé (mot), associe la liste des mots qui le suivent dans tout le corpus. Il suffit donc de compter l'occurence de chaque mot dans cette liste pour calculer les probabilités de transition d'un mot à un autre dans le texte. Ainsi, il très facile de générer un nouveau texte (ou article) à partir de ces probabilités de transition. 

La vidéo suivante explique très bien comment procéder facilement de A à Z à la création d'une chaine markov pour notre corpus d'articles : https://www.youtube.com/watch?v=MGVdu39gT6k

### 2.2. Pros and Cons

*Pros* :
   * Il s'agit d'un modèle très simple et facile à implémenter (basé essentiellement sur comptage de mots)
   * Le temps d'entrainement est négligeable


*Cons* :
   * Très peu performant, car peut facilement générer un texte dépourvu de sens

## 3. Des réseaux de neurones basiques : le LSTM

De son nom complet "**Long Short Term Memory**", il s'agit d'un réseau de neurone réccurrent en deep learning (les connexions entre les neurones sont réccurents) très utilisé dans les études de series temporelles pour faire de la prévision. 

### 3.1. Cas de la génération d'articles

Comme dans le cas des chaines de Markov vus précédemment, le LSTM est très bien adapté à ce type de problème puisqu'elle fait aussi de la prédiction de lettre suivante à partir des précédentes. Il existe une énorme quantité de documents, de repositories git, d'articles sur le LSTM en ligne (il y en a vraiment beaucoup, on peut facilement trouver du contenu intéressant selon notre besoin). Un des articles qui pourrait nous aider : https://www.analyticsvidhya.com/blog/2018/03/text-generation-using-python-nlp/ 

### 3.2. Pros et Cons

*Pros* :
   * C'est un modèle extrêment personnalisable, plusieurs paramètres à tuner...
   * Les performances peuvent être nettement mieux comparé à celle d'une chaine de markov si jamais la phase d'entrainement bien faite
   * Possibilité d'utilisé des modèles préentrainés


*Cons* :
   * Peut prendre du temps si on veut entrainer son propre corpus

## 4. Un nouvel état de l'art du NLP : le GPT-2 de OpenAI

Le traitement de langage naturel reste en constante innovation. Le modèle GPT-2 de OpenAI a récemment éclaté les performances en terme de génération automatique de texte, le tout avec une rapidité et une efficacité hors norme. Plusieurs articles et vidéos sont disponibles pour expliquer l'architecture et le fonctionnement de GPT-2 qui utilise des *transformers* comme réseau de neurone. 

### 4.1. Du transfert learning pour la génération d'articles similaires

Plusieurs versions préentrainés de GPT-2 existent, seules deux versions ont été publiées. Il est possible d'utiliser l'une des deux versions pour faire du transfer learning (apprentissage par transfert consistant à utiliser une majeure partie des poids présents dans le réseau déjà entrainé en réajustant les poids de certaines couches finales à un dataset particulier) en utilisant les textes des articles que nous avons dans ce notebook. Pour faire celà, il y a un excellent article : https://minimaxir.com/2019/09/howto-gpt2/

L'éxécution de bout en bout de cet article nécessitant plusieurs fonctionnalités non disponibles en local dans "jupyter", j'ai préféré me limiter à la bonne compréhension de ce dernier qui peut facilement être réadapté à notre contexte. Il présente la particularité de pouvoir bien choisir son corpus en l'incluant dans la phase d'apprentissage.

### 4.2. Utilisation directe d'un modèle GPT-2

Il est également possible d'utiliser directement un modèle GPT-2 publié par OpenAI pour faire de la prédiction. On peut alors changer les paramètres du modèle préentrainé à notre guise (length, temperature, top_k, sont des paramètres que nous pouvons changer nous même) avec le déjà utilisé par GPT-2. Il n'est pas garanti que les résultats soient satisfaisantes, mais comme GPT-2 a été entrainé sur des millions de pages web scrappées, il est bien probable que le contenu généré soit de très bonne qualité. Voici un article intéressant si cette méthode nous intéresse : https://www.analyticsvidhya.com/blog/2019/07/openai-gpt2-text-generator-python/